# RAG with Kendra

#### If you are running this in sagemaker studio you need to select a kernel with a python version >3.8

このノートブックでは、rinna 社の japanese-gpt-neox-3.6b-instruction-ppo モデルを、SageMaker のリアルタイム推論エンドポイントとして Hosting し、Amazon Kendra で取得したドキュメントを要約して返す RAG (Retrieval Augmented Generation) を体験します。
以下の環境で動作確認を行なっています。

- SageMaker Studio Notebooks
  - `ml.m5.2xlarge (RAM 32GB, vCPU 8)` : `PyTorch 1.13 Python 3.9 CPU Optimized`
  
前半の rinna をデプロイするパートは [aws-ml-jp で公開しているノートブック](https://github.com/aws-samples/aws-ml-jp/blob/main/tasks/generative-ai/text-to-text/inference/deploy-endpoint/Transformers/rinna-3.6b-instruction-ppo_Inference.ipynb.ipynb)の手順に基づいています。


---

This notebook is based on the this aws blog 
* https://aws.amazon.com/blogs/machine-learning/quickly-build-high-accuracy-generative-ai-applications-on-enterprise-data-using-amazon-kendra-langchain-and-large-language-models/

---

### Solution overview
The following diagram shows the architecture of a GenAI application with a RAG approach.

<img src="https://d2908q01vomqb2.cloudfront.net/f1f836cb4ea6efb2a0b1b99f41ad8b103eff4b59/2023/05/02/ML-13807-image001-new.png">

We use an Amazon Kendra index to ingest enterprise unstructured data from data sources such as wiki pages, MS SharePoint sites, Atlassian Confluence, and document repositories such as Amazon S3. When a user interacts with the GenAI app, the flow is as follows:

1. The user makes a request to the GenAI app.
2. The app issues a search query to the Amazon Kendra index based on the user request.
3. The index returns search results with excerpts of relevant documents from the ingested enterprise data.
4. The app sends the user request and along with the data retrieved from the index as context in the LLM prompt.
5. The LLM returns a succinct response to the user request based on the retrieved data.
6. The response from the LLM is sent back to the user.

With this architecture, you can choose the most suitable LLM for your use case. LLM options include our partners Hugging Face, AI21 Labs, Cohere, and others hosted on an Amazon SageMaker endpoint, as well as models by companies like Anthropic and OpenAI. With Amazon Bedrock, you will be able to choose Amazon Titan, Amazon’s own LLM, or partner LLMs such as those from AI21 Labs and Anthropic with APIs securely without the need for your data to leave the AWS ecosystem. The additional benefits that Amazon Bedrock will offer include a serverless architecture, a single API to call the supported LLMs, and a managed service to streamline the developer workflow.

For the best results, a GenAI app needs to engineer the prompt based on the user request and the specific LLM being used. Conversational AI apps also need to manage the chat history and the context. GenAI app developers can use open-source frameworks such as LangChain that provide modules to integrate with the LLM of choice, and orchestration tools for activities such as chat history management and prompt engineering. We have provided the KendraIndexRetriever class, which implements a LangChain retriever interface, which applications can use in conjunction with other LangChain interfaces such as chains to retrieve data from an Amazon Kendra index. We have also provided a few sample applications in the GitHub repo. You can deploy this solution in your AWS account using the step-by-step guide in this post.

---

0. [Prerequisites](#Prerequisites)
1. [Permissions and environment variables](#1.-Permissions-and-environment-variables)
2. [Select a pre-trained model](#2.-Select-a-pre-trained-model)
3. [Retrieve Artifacts & Deploy an Endpoint](#3.-Retrieve-Artifacts-&-Deploy-an-Endpoint)
4. [Query endpoint and parse response](#4.-Query-endpoint-and-parse-response)
5. [Query endpoint with Langchain and Kendra Index](#5.-Query-endpoint-with-Langchain-and-Kendra-Index)
6. [[OPTIONAL] Installing Streamlet application and running a WebUI for a chatbot](#6.-[OPTIONAL]-Installing-Streamlit-application-and-running-a-WebUI-for-a-chatbot)
7. [Clean up the endpoint](#7.-Clean-up-the-endpoint)

---

## Prerequisites

### Kendra Index

---

Use the provided AWS CloudFormation to create a new Amazon Kendra index. This template includes sample data containing AWS online documentation for Amazon Kendra, Amazon Lex, and Amazon SageMaker. Alternately, if you have an Amazon Kendra index and have indexed your own dataset, you can use that. 


Deployment steps:
   1. Download the [template](https://github.com/kmotohas/amazon-kendra-langchain-extensions/blob/main/kendra_retriever_samples/kendra-docs-index.yaml) from the github repo
   2. Deploy it using the [cloudformation console](https://us-east-1.console.aws.amazon.com/cloudformation/home?region=us-east-1#/stacks/create)
      1. Select Upload a template file and then the choose file button to select the template you just downloaded
      2. Press Next
   3. Provide a stack name and press Next   
   4. Leave all default options and press Next
   5. Check the acknowledgement at the bottom and press Submit
   6. The stack will take around 15 minutes to deploy
   7. Take note of the `AWSRegion` and `KendraIndexID` in the Outputs tab as we will need it in later steps   
---

### Kendra Permissions
---

You will need to update your SageMaker execution role with permissions to query Kendra. 

1. Navigate to IAM console and select Roles
2. Search for your SageMaker execution role it will look like `AmazonSageMaker-ExecutionRole-<TIMESTAMP>`
3. Select your role and add permission search for `AmazonKendraReadOnlyAccess` and attach the policy

---

### AWS Langchain

---

This repo provides a set of utility classes to work with Langchain. It currently has a retriever class KendraIndexRetriever for working with a Kendra index and sample scripts to execute the QA chain for SageMaker, Open AI and Anthropic providers.

---

Clone the repository:

In [2]:
#!git clone https://github.com/aws-samples/amazon-kendra-langchain-extensions.git

Cloning into 'amazon-kendra-langchain-extensions'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 75 (delta 38), reused 22 (delta 13), pack-reused 19
Receiving objects: 100% (75/75), 25.33 KiB | 741.00 KiB/s, done.
Resolving deltas: 100% (39/39), done.


Install the classes:
we use a specific version of amazon-kendra-langchain-extensions as it later gets integrated into langchain and would require change of the code of this lab

In [3]:
#!cd amazon-kendra-langchain-extensions && git checkout 28cb1d4de7cf3bfe8984c6365ce248c12e8b77e0 && pip install . --quiet

Note: switching to '28cb1d4de7cf3bfe8984c6365ce248c12e8b77e0'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 28cb1d4 Don't take a source if it is already present. Fixed hardcoding of region
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.3.2 which is incompatible.


## 1. 準備

### ノートブックを動かすに当たって必要なモジュールのインストール

In [96]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
!pip install -U pip sagemaker boto3 transformers==4.26 einops SentencePiece langchain ipywidgets "pydantic<2" streamlit

  Using cached streamlit-1.24.1-py2.py3-none-any.whl (8.9 MB)
  Using cached altair-5.0.1-py3-none-any.whl (471 kB)
  Using cached blinker-1.6.2-py3-none-any.whl (13 kB)
  Using cached cachetools-5.3.1-py3-none-any.whl (9.3 kB)
  Using cached Pympler-1.0.1-py3-none-any.whl (164 kB)
  Using cached tzlocal-4.3.1-py3-none-any.whl (20 kB)
  Using cached validators-0.20.0-py3-none-any.whl
  Using cached GitPython-3.1.32-py3-none-any.whl (188 kB)
  Using cached pydeck-0.8.1b0-py2.py3-none-any.whl (4.8 MB)
  Using cached watchdog-3.0.0-py3-none-manylinux2014_x86_64.whl (82 kB)
  Using cached gitdb-4.0.10-py3-none-any.whl (62 kB)
  Using cached pytz_deprecation_shim-0.1.0.post0-py2.py3-none-any.whl (15 kB)
  Using cached smmap-5.0.0-py3-none-any.whl (24 kB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)


### 今回扱うモデルの動かし方について
[How to use the model](https://huggingface.co/rinna/japanese-gpt-neox-3.6b-instruction-ppo#how-to-use-the-model) に沿って実行すると動かせます。
例えば、以下のコードをこのノートブックで実行するとテキストを生成できます。
実行したい場合は別途セルを用意して実行してみてください。g5.2xlarge インスタンスで実行に 10 分程度かかります。(ほとんどはモデルのロード時間です)
このノートブックでは以下のコードをベースに SageMaker で Hosting できるようにします。

```python
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(
    'rinna/japanese-gpt-neox-3.6b-instruction-ppo', 
    use_fast=False
)
model = AutoModelForCausalLM.from_pretrained(
    'rinna/japanese-gpt-neox-3.6b-instruction-ppo'
).to("cuda")

prompt = '''ユーザー: 世界自然遺産を列挙してください。
システム: 膨大な数です。例えば国で絞ってください。
ユーザー: イギリスでお願いします。
システム:'''.replace('\n','<NL>')

token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        do_sample=True,
        max_new_tokens=128,
        temperature=0.01,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1):])
output = output.replace("<NL>", "\n")
print(output)
```

### モジュール読み込み

In [5]:
# import sagemaker, boto3, json
# from sagemaker.session import Session

# sagemaker_session = Session()
# aws_role = sagemaker_session.get_caller_identity_arn()
# aws_region = boto3.Session().region_name
# sess = sagemaker.Session()

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import gc
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
import json
region = boto3.session.Session().region_name
role = sagemaker.get_execution_role()
sm = boto3.client('sagemaker')
smr = boto3.client('sagemaker-runtime')
endpoint_inservice_waiter = sm.get_waiter('endpoint_in_service')

### モデルのダウンロード
SageMaker で機械学習モデルをホスティングする際は、一般的にはモデルや推論コードなどを tar.gz の形に固めます。
tokenizer と model を `from_pretrained` メソッドを利用してモデルをインターネットからロードして、そのままファイルをディレクトリに出力します。

In [8]:
# 既存のディレクトリがある場合のときのため削除
model_dir = './inference'
!rm -rf {model_dir}
!mkdir -p {model_dir}'/code'

### tokenizer の取得と保存

In [9]:
%%time

tokenizer = AutoTokenizer.from_pretrained(
    'rinna/japanese-gpt-neox-3.6b-instruction-ppo', 
    use_fast=False
)
tokenizer.save_pretrained(model_dir)

CPU times: user 160 ms, sys: 11.9 ms, total: 172 ms
Wall time: 574 ms


('./inference/tokenizer_config.json',
 './inference/special_tokens_map.json',
 './inference/spiece.model',
 './inference/added_tokens.json')

### モデルの取得と保存

以下のセルは 10GB 以上のモデルを DL して保存するため 5 分ほど時間がかかります。

In [10]:
%%time
model = AutoModelForCausalLM.from_pretrained(
    'rinna/japanese-gpt-neox-3.6b-instruction-ppo'
)
model.save_pretrained(model_dir)

CPU times: user 42 s, sys: 29.6 s, total: 1min 11s
Wall time: 4min 34s


モデルは SageMaker で動かすのでメモリから開放します

In [11]:
del model
del tokenizer
gc.collect()

259

### 推論コードの作成
先程実行したコードをもとに記述していきます。
まずは必要なモジュールを記述した requirements.txt を用意します。
今回は [deep-learning-containers](https://github.com/aws/deep-learning-containers)の HuggingFace のコンテナを使います。
einops と Sentence Piece が不足しているので requirements.txt に記載します。

In [12]:
%%writefile inference/code/requirements.txt
einops
SentencePiece

Writing inference/code/requirements.txt


先述のコードを SageMaker Inference 向けに改変します。

1. `model_fn` でモデルを読み込みます。先程は huggingface のモデルを直接ロードしましたが、model_dir に展開されたモデルを読み込みます。
2. `input_fn` で前処理を行います。
  - json 形式のみを受け付け他の形式は弾くようにします。
  - json 文字列を dict 形式に変換して返します。
3. `predict_fn` で推論します。
  - リクエストされたテキストを token 化します。
  - パラメータを展開します。
  - 推論（生成）します。
  - 生成結果をテキストにして返します。
4. `output_fn` で結果を json 形式にして返します。


In [13]:
%%writefile inference/code/inference.py
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import json

DEVICE = 'cuda:0'

def model_fn(model_dir):
    tokenizer = AutoTokenizer.from_pretrained(
        model_dir, 
        use_fast=False
    )
    model = AutoModelForCausalLM.from_pretrained(
        model_dir
    ).to(DEVICE)
    return {'tokenizer':tokenizer,'model':model}

def input_fn(data, content_type):
    if content_type == 'application/json':
        data = json.loads(data)
    else:
        raise TypeError('content_type is only allowed application/json')
    return data

def predict_fn(data, model):
    prompt = data['prompt']
    token_ids = model['tokenizer'].encode(prompt, add_special_tokens=False, return_tensors="pt")
    do_sample = data['do_sample']
    max_new_tokens = data['max_new_tokens']
    temperature = data['temperature']
    
    with torch.no_grad():
        output_ids = model['model'].generate(
            token_ids.to(DEVICE),
            do_sample=do_sample,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            pad_token_id=model['tokenizer'].pad_token_id,
            bos_token_id=model['tokenizer'].bos_token_id,
            eos_token_id=model['tokenizer'].eos_token_id
        )
    output = model['tokenizer'].decode(output_ids.tolist()[0][token_ids.size(1):])
    output = output.replace("<NL>", "\n")
    
    return output


def output_fn(data, accept_type):
    if accept_type == 'application/json':
        data = json.dumps({'result' : data})
    else:
        raise TypeError('content_type is only allowed application/json')
    return data

Writing inference/code/inference.py


## モデルアーティファクトの作成と S3 アップロード
アーティファクト(推論コード + モデル)を tar.gz に固めます。時間がかかるので `pigz` で並列処理を行います。
ml.g5.2xlarge, ml.m5.2xlarge で 10 分ほどかかります。

※ SageMaker Studio のカーネルには pigz が入っていないので、下記 apt のセルを実行してください。SageMaker Notebooks の場合は不要です。

In [16]:
!apt update -y
!apt install pigz -y

Get:1 http://archive.ubuntu.com/ubuntu focal InRelease [265 kB]
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:4 http://security.ubuntu.com/ubuntu focal-security/multiverse amd64 Packages [37.5 kB]
Get:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:6 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1070 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal/restricted amd64 Packages [33.4 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal/universe amd64 Packages [11.3 MB]
Get:9 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2866 kB]
Get:10 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [2529 kB]3m
Get:11 http://archive.ubuntu.com/ubuntu focal/main amd64 Packages [1275 kB]    
Get:12 http://archive.ubuntu.com/ubuntu focal/multiverse amd64 Packages [177 kB]
Get:13 http://archive.ubuntu.com/ubu

In [17]:
%%time

!rm model.tar.gz
%cd {model_dir}
!tar  cv ./ | pigz -p 8 > ../model.tar.gz # 8 並列でアーカイブ
%cd ..

rm: cannot remove 'model.tar.gz': No such file or directory
/root/apj-generative-ai-l300-bootcamp-main/Day2/inference
./
./pytorch_model-00002-of-00002.bin
./spiece.model
./tokenizer_config.json
./config.json
./pytorch_model.bin.index.json
./generation_config.json
./special_tokens_map.json
./code/
./code/requirements.txt
./code/inference.py
./pytorch_model-00001-of-00002.bin
/root/apj-generative-ai-l300-bootcamp-main/Day2
CPU times: user 8.69 s, sys: 1.04 s, total: 9.73 s
Wall time: 9min 53s


アーティファクトを S3 にアップロードします。60 秒程度で完了します。

In [18]:
%%time

model_s3_uri = sagemaker.session.Session().upload_data(
    'model.tar.gz',
    key_prefix='japanese-gpt-neox-3.6b-instruction-ppo'
)
print(model_s3_uri)

s3://sagemaker-us-east-1-809078683005/japanese-gpt-neox-3.6b-instruction-ppo/model.tar.gz
CPU times: user 46.7 s, sys: 48.2 s, total: 1min 34s
Wall time: 33.9 s


## SageMaker で Hosting する
g5.2xlarge インスタンス(NVIDIA A10G Tensor Core GPU 搭載 VRAM 24GB, RAM 32GB) の場合レスポンスに 6 秒程度で済むため、リアルタイム推論エンドポイントを立てます。
[g5.2xlarge の料金はこちら](https://aws.amazon.com/sagemaker/pricing/?nc1=h_ls)で確認してください。

<!--リアルタイム推論エンドポイントを立てて推論するにあたって、SageMaker Python SDK を用いる場合と Boto3 を用いる場合の 2 パターンを紹介します。-->

### SageMaker Python SDKを用いる場合
#### Hosting
使用している API の詳細は以下を確認してください。
Amazon SageMaker Python SDK

#### 定数の設定

In [6]:
model_name = 'japanese-gpt-neox-3-6b-instruction-ppo'
endpoint_config_name = model_name + '-Config'
endpoint_name = model_name + '-Endpoint'
instance_type = 'ml.g5.2xlarge'

#### 使用するコンテナイメージの URI を取得

In [7]:
image_uri = sagemaker.image_uris.retrieve(
    framework='huggingface',
    region=region,
    version='4.26',
    image_scope='inference',
    base_framework_version='pytorch1.13',
    instance_type = instance_type
)

#### モデルの定義
先程 S3 にアップロードしたアーティファクトの tar.gz の URI と、コンテナイメージの URI, ロールを設定します。

In [8]:
huggingface_model = HuggingFaceModel(
    model_data = model_s3_uri,
    role = role,
    image_uri = image_uri
)

NameError: name 'model_s3_uri' is not defined

#### デプロイ

In [22]:
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    endpoint_name=endpoint_name,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer()
)

----------!

In [114]:
endpoint_name = predictor.endpoint_name
endpoint_name

'japanese-gpt-neox-3-6b-instruction-ppo-Endpoint'

In [113]:
%store endpoint_name

Stored 'endpoint_name' (str)


### 推論
#### promptについて
[japanese-gpt-neox-3.6b-instruction-ppo#io-format](https://huggingface.co/rinna/japanese-gpt-neox-3.6b-instruction-ppo#io-format) にある通り、以下の通りにすると良い結果が得られやすいです。

- プロンプトはユーザーとシステムの会話形式で与える
- 各発言は、以下形式に則る
`{ユーザー, システム} : {発言}`
- プロンプトの末尾は `システム:` で終了させる
- 改行は `<NL>` を利用し、発言はすべて `<NL>` で区切る必要がある
以下はプロンプトの例です。`<NL>` の埋め込みが大変なので、改行で書いて後で置換します。

In [23]:
prompt = '''ユーザー: 世界自然遺産を列挙してください。
システム: 膨大な数です。例えば国で絞ってください。
ユーザー: イギリスでお願いします。
システム:'''.replace('\n','<NL>')
print(prompt)

ユーザー: 世界自然遺産を列挙してください。<NL>システム: 膨大な数です。例えば国で絞ってください。<NL>ユーザー: イギリスでお願いします。<NL>システム:


#### 推論リクエスト
model_fn の実行に時間がかかってしまい、エンドポイントが IN_SERVICE になっても、初回推論はしばらく動かないことがあります。
CloudWatch Logs に以下のような表示がある場合はしばらく待てば使えるようになります。
> `[WARN] pool-3-thread-1 com.amazonaws.ml.mms.metrics.MetricCollector - worker pid is not available yet.`

モデルがロードされるまで 6 分程度かかるため、リトライを入れています。 実際の推論時間は 6 秒程度です。

In [28]:
from time import sleep
request = {
    'prompt' : prompt,
    'max_new_tokens' : 256,
    'do_sample' : True,
    'temperature' : 0.01,
}

for i in range(10):
    try:
        output = predictor.predict(request)['result']
        break
    except:
        sleep(60)
print(output)

イギリスには、スコットランド、ウェールズ、イングランド、北アイルランドの世界遺産があります。これらは、世界で最も古い自然遺産のいくつかです。</s>


In [20]:
question = "Amazon Kendraはどのデータソースから回答を返してくれますか？"
#question = "犬は何を食べますか？"
context = "Amazon Kendra は、機械学習 (ML) を利用する高精度で使いやすいエンタープライズ検索サービスです。" \
"デベロッパーはアプリケーションに検索機能を追加できます。" \
"これにより、その企業全体に散在する膨大な量のコンテンツ内に保存されている情報をエンドユーザーが見つけられるようになります。" \
"これには、マニュアル、調査報告書、よくある質問、人事 (HR) 関連ドキュメント、カスタマーサービスガイドのデータが含まれます。" \
"Amazon Simple Storage Service (S3)、Microsoft SharePoint、Salesforce、ServiceNow、RDS データベース、Microsoft OneDrive などの様々なシステムに存在している場合があります。" \
"質問が入力されると、このサービスは機械学習アルゴリズムを使用してその内容を理解し、質問の直接の回答であれ、ドキュメント全体であれ、最も適切な回答を返します。" \
"例えば、「企業クレジットカードのキャッシュバック率はどれくらいですか?」といった質問をすることができ、Amazon Kendra は関連するドキュメントにマッピングして具体的な回答 (「2% です」など) を返します。" \
"Kendra はサンプルコードを提供するため、ユーザーは迅速に使用を開始し、新規または既存のアプリケーションに極めて正確な検索を簡単に統合できます。"
prompt = f"""
システム: システムは資料から抜粋して質問に答えます。資料にない内容は答えず、正直に「わかりません」と答えます。

{context}

上記の資料に基づいて以下の質問について資料から抜粋して回答を生成します。資料にない内容は答えず「わかりません」と答えます。
ユーザー: {question}
""".replace("\n", "<NL>")
print(prompt)


<NL>システム: システムは資料から抜粋して質問に答えます。資料にない内容は答えず、正直に「わかりません」と答えます。<NL><NL>Amazon Kendra は、機械学習 (ML) を利用する高精度で使いやすいエンタープライズ検索サービスです。デベロッパーはアプリケーションに検索機能を追加できます。これにより、その企業全体に散在する膨大な量のコンテンツ内に保存されている情報をエンドユーザーが見つけられるようになります。これには、マニュアル、調査報告書、よくある質問、人事 (HR) 関連ドキュメント、カスタマーサービスガイドのデータが含まれます。Amazon Simple Storage Service (S3)、Microsoft SharePoint、Salesforce、ServiceNow、RDS データベース、Microsoft OneDrive などの様々なシステムに存在している場合があります。質問が入力されると、このサービスは機械学習アルゴリズムを使用してその内容を理解し、質問の直接の回答であれ、ドキュメント全体であれ、最も適切な回答を返します。例えば、「企業クレジットカードのキャッシュバック率はどれくらいですか?」といった質問をすることができ、Amazon Kendra は関連するドキュメントにマッピングして具体的な回答 (「2% です」など) を返します。Kendra はサンプルコードを提供するため、ユーザーは迅速に使用を開始し、新規または既存のアプリケーションに極めて正確な検索を簡単に統合できます。<NL><NL>上記の資料に基づいて以下の質問について資料から抜粋して回答を生成します。資料にない内容は答えず「わかりません」と答えます。<NL>ユーザー: Amazon Kendraはどのデータソースから回答を返してくれますか？<NL>


In [21]:
request = {
    'prompt' : prompt,
    'max_new_tokens' : 256,
    'do_sample' : True,
    'temperature' : 0.7,
    'repetition_penalty': 1.3,
}
output = predictor.predict(request)['result']
print(output)

Amazon Simple Storage Service (S3)、Microsoft SharePoint、Salesforce、RDS データベース、Microsoft OneDrive などの様々なシステムに存在している場合があります。回答は、機械学習アルゴリズムを使用して、関連するドキュメントにマッピングし、適切な回答を返します。</s>


#### エンドポイントの削除

In [ ]:
# predictor.delete_model()
# predictor.delete_endpoint()

In [3]:
#model_id, model_version = "huggingface-text2text-flan-t5-xl", "*"
#model_id, model_version = "huggingface-textgeneration1-mpt-7b-instruct-bf16", "*"
model_id, model_version = "huggingface-llm-falcon-7b-instruct-bf16", "*"

### 5. Query endpoint with Langchain and Kendra Index

---
Now we will use use the `KendraIndexRetriever` retriever class with Langchain to retrieve information from our Kendra Index that matches the query.

---

In [72]:
!aws kendra list-indices

{
    "IndexConfigurationSummaryItems": [
        {
            "Name": "demo-index",
            "Id": "c9f7203c-bad2-45bf-a0ed-dbaf0e201e93",
            "Edition": "DEVELOPER_EDITION",
            "CreatedAt": 1678180374.447,
            "UpdatedAt": 1687854347.072,
            "Status": "ACTIVE"
        }
    ]
}


In [9]:
# UPDATE THE FOLLOWING WITH THE OUTPUTS FROM THE CLOUDFORMATION DEPLOYMENT
kendra_index_id="c9f7203c-bad2-45bf-a0ed-dbaf0e201e93"
region="us-east-1"

In [10]:
%store -r endpoint_name
endpoint_name

'japanese-gpt-neox-3-6b-instruction-ppo-Endpoint'

In [11]:
from sagemaker.huggingface import HuggingFacePredictor
predictor = HuggingFacePredictor(endpoint_name=endpoint_name)

In [31]:
newline, bold, unbold = "\n", "\033[1m", "\033[0m"

In [45]:
#from aws_langchain.kendra_index_retriever import KendraIndexRetriever
from langchain.retrievers import AmazonKendraRetriever
from langchain.chains import RetrievalQA
from langchain import OpenAI
from langchain.prompts import PromptTemplate
from langchain import SagemakerEndpoint
from langchain.llms.sagemaker_endpoint import LLMContentHandler
import json

class RinnaContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs: dict) -> bytes:
        input_str = json.dumps(
            {
                "instruction": "",
                #"input": prompt.replace("\n", "<NL>"),
                "prompt": prompt.replace("\n", "<NL>"),
                **model_kwargs,
            }
        )
        print("prompt: ", prompt)
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        #return response_json.replace("<NL>", "\n")
        #print(response_json)
        return response_json["result"]

#     def transform_input(self, prompt: str, model_kwargs: dict) -> bytes:
#         input_str = json.dumps({"text_inputs": prompt, **model_kwargs})
#         return input_str.encode('utf-8')

#     def transform_output(self, output: bytes) -> str:
#         response_json = json.loads(output.read().decode("utf-8"))
#         return response_json["generated_texts"][0]

content_handler = RinnaContentHandler()
llm=SagemakerEndpoint(
        endpoint_name=endpoint_name,
        region_name="us-east-1", 
        model_kwargs={
            "temperature":1e-10,
            "max_new_tokens": 256,
            "do_sample": True,
            "temperature" : 0.7,
            "repetition_penalty": 1.3,
        },
        content_handler=content_handler
    )

#retriever = KendraIndexRetriever(kendraindex=kendra_index_id,
retriever = AmazonKendraRetriever(
        index_id=kendra_index_id,
        region_name=region,
        top_k=3,
        credentials_profile_name=None,
        attribute_filter={
            "EqualsTo": {      
                "Key": "_language_code",
                "Value": {
                    "StringValue": "ja"
                    }
                }
        }
#        return_source_documents=True
    )

# prompt_template = """
# The following is a friendly conversation between a human and an AI.
# The AI is talkative and provides lots of specific details from its context.
# If the AI does not know the answer to a question, it truthfully says it
# does not know.
# {context}
# Instruction: Based on the above documents, provide a detailed answer for, {question} Answer "don't know" if not present in the document. Solution:
# """
prompt_template = """
システム: システムは資料から抜粋して質問に答えます。資料にない内容には答えず、正直に「わかりません」と答えます。

{context}

上記の資料に基づいて以下の質問について資料から抜粋して回答を生成します。資料にない内容には答えず「わかりません」と答えます。
ユーザー: {question}
システム:
"""#.replace("\n", "<NL>")
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(
    llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs=chain_type_kwargs,
    return_source_documents=True
)

result = qa("ワークロードの可用性を上げたい")
#result = qa("犬かわいすぎ")
print(f'{bold}Answer{unbold}: {result["result"]}\n\n{bold}Sources:{unbold}')

for doc in result['source_documents']:
    print(f'''\n{doc.metadata["title"]}\n{doc.metadata["source"]}\n{doc.metadata["excerpt"]}\n''')


prompt:  
システム: システムは資料から抜粋して質問に答えます。資料にない内容には答えず、正直に「わかりません」と答えます。

Document Title: wellarchitected-reliability-pillar 
Document Excerpt: 
• REL02-BP04 多対多メッシュよりもハブアンドスポークトポロジを優先する (p. 36) • REL02-BP05 接続されているすべてのプライベートアドレス空間において、重複しないプライベート IP アドレス範囲を指定する (p. 38) REL02-BP01 ワークロードのパブリックエンドポイン トに高可用性ネットワーク接続を使用する ワークロードのパブリックエンドポイントに高可用性ネットワーク接続を構築すると、接続の喪失による ダウンタイムを低減し、ワークロードの可用性と SLA を向上できます。 これを実現するには、可用性の高 い DNS、コンテンツ配信ネットワーク、API ゲートウェイ、負荷分散、またはリバースプロキシを使用し ます。 期待される成果: パブリックエンドポイントの高可用性ネットワーク接続を計画、構築、運用化すること が重要です。 接続が失われたためにワークロードにアクセスできなくなった場合、ワークロードが実行中 28 http://aws.amazon.com/codedeploy/ http://aws.amazon.com/cloudtrail/ http://aws.amazon.com/cloudwatch/ http://aws.amazon.com/eventbridge/ http://aws.amazon.com/devops-guru/ http://aws.amazon.com/config/ http://aws.amazon.com/premiumsupport/technology/trusted-advisor/ http://aws.amazon.com/cdk/ http://aws.amazon.com/systems-manager/


Document Title: 可用性ニーズの理解 - 信頼性の柱 
Document Excerpt: 
AWS のソリューションアーキテクト (SA) は、お客様の可用性目標に対す

### Example Queries

The quality of the response you will recieve is going to depend on the model you have deployed. Try deploying a different model endpoint and comparing the results you recieve from the same queries.

In [ ]:
result = qa("ワークロードの可用性を上げたい")
print(f'{bold}Answer{unbold}: {result["result"]}')

In [42]:
result = qa("AWSの信頼性を高める機能についてを教えてください。")
print(f'{bold}Answer{unbold}: {result["result"]}')

prompt:  
システム: システムは資料から抜粋して質問に答えます。資料にない内容には答えず、正直に「わかりません」と答えます。

Document Title: wellarchitected-reliability-pillar 
Document Excerpt: 
関連動画: • AWS の静的安定性: AWS re:Invent 2019: Introducing The Amazon Builders’ Library (DOP328) REL11-BP06 イベントが可用性に影響する場合に通知 を送信する 重大なイベントが検出されると、イベントによって引き起こされた問題が自動的に解決された場合でも、 通知が送信されます。 自動ヒーリング機能により、ワークロードの信頼性を高めることができます。 ただし、対処する必要のあ る根本的な問題もあいまいになる可能性があります。 根本原因の問題を解決できるように、自動ヒーリ ングによって対処されたものを含む問題のパターンを検出できるように、適切なモニタリングとイベン トを実装します。 Amazon CloudWatch アラームは、発生した障害に基づいてトリガーできます。 また、 実行された自動ヒーリングアクションに基づいてトリガーすることもできます。 CloudWatch アラーム は、Amazon SNS 統合を使用して、E メールを送信するか、サードパーティのインシデント追跡システム


Document Title: wellarchitected-reliability-pillar 
Document Excerpt: 
が高いため、許可すべきではありません。 リソース 関連するドキュメント: • 災害対策プランでの依存関係の最小化 • The Amazon Builders' Library: アベイラビリティーゾーンを使用した静的安定性 関連動画: • AWS の静的安定性: AWS re:Invent 2019: Introducing The Amazon Builders’ Library (DOP328) REL11-BP06 イベントが可用性に影響する場合に通知 を送信する 重大なイベントが検出されると、イベントによって引き起こされた問題が自動的に解決された場合でも、 通知が送信され

### 6. [OPTIONAL] Installing Streamlit application and running a WebUI for a chatbot

---
This sections provides instructions on how to run a streamlet application within sagemaker studio and accessing it using jupyter proxy. The commands and instructions below need to be run inside a **SageMaker System Terminal**.

---



In [ ]:
!git clone https://github.com/kmotohas/amazon-kendra-langchain-extensions.git

1. Launch a new SageMaker System Terminal 
   1. From the SageMaker Studio Home screen select `Open Launcher`
   2. From the Launcher panel under `Utilities and files` select `System terminal`

5. Set your environment variables
```
export AWS_REGION="<YOUR_AWS_REGION>"
export KENDRA_INDEX_ID="<YOUR_KENDRA_INDEX_ID>"
export RINNA_ENDPOINT="<YOUR_SAGEMAKER_ENDPOINT_FOR_RINNA>"
```
6. Run the streamlit application
```
cd ./amazon-kendra-langchain-extensions/kendra_retriever_samples
streamlit run app.py rinna
```
7. This will output something similar to the below, you need to take note of the port (in this case 8501)
```
Collecting usage statistics. To deactivate, set browser.gatherUsageStats to False.


  You can now view your Streamlit app in your browser.

  Network URL: http://169.255.255.2:8501
  External URL: http://18.213.200.192:8501
```
8. Copy the current URL of the SageMaker Studio which should have the form:
```
https://<YOUR_STUDIO_DOMAIN>.studio.<AWS_REGION>.sagemaker.aws/jupyter/default/lab/workspaces/auto-Z/tree/kendra_rag_demo.ipynb
```
9. Delete everything from `lab/` onwards and replace it with `proxy/<PORT>/`
   1. DON'T FORGET THE END `/`
```
https://<YOUR_STUDIO_DOMAIN>.studio.<AWS_REGION>.sagemaker.aws/jupyter/default/proxy/8501/
```
10. Paste the new address into the browser and you will now be able to access your chatbot UI which uses Langchain and Kendra. Each response will list the sources from Kendra it used for its answers.

### 7. Clean up the endpoint

In [77]:
# Delete the SageMaker endpoint
predictor.delete_model()
predictor.delete_endpoint()